# LASSO (coordinate descent)

Trong notebook này, chúng ta sẽ triển khai bộ giải LASSO qua coordinate descent. Chúng ta sẽ:
* Viết một hàm chuẩn hóa các đặc trưng
* Triển khai coordinate descent cho LASSO
* Khám phán tác động của L1 penalty

## Như thường lệ

In [34]:
import sklearn
import pandas
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## Load dữ liệu doanh số bán nhà

Tập dữ liệu doanh số bán nhà ở quận King, Seatle, WA. Nghe quen chứ?

In [35]:
full_data = pandas.read_csv("house_data/kc_house_data.csv", index_col=0)
# Trong tập dữ liệu, 'floors' được xác định là kiểu string,
# nên chúng ta sẽ chuyển đổi chúng thành int trước khi sử dụng dưới đây
full_data['floors'] = full_data['floors'].astype(int) 

Nếu muốn thực hiện bất kỳ "feature engineering" nào như tạo các đặc trưng mới hoặc điều chỉnh đặc trưng sẵn có, chúng ta có thể sửa DataFrame của pandas như trong lab trước (Lab 2). Tuy nhiên, với notebook này, chúng ta sẽ làm việc với các đặc trưng có sẵn.

## Import các hàm hữu ích từ notebook trước

Như trong Exercise 1, chúng ta chuyển đổi DataFrame thành ma trận Numpy 2D. Copy và paste `get_numpy_data()` từ exercise đó.

In [36]:
import numpy as np # điều này cho phép gọi numpy as np 

In [37]:
def get_numpy_data(data, features_title, labels_title):
    if('constant' not in data):
        data['constant'] = 1 # đây là cách thêm cột constant. Chỉ thực hiện khi cần 
    # thêm cột 'constant' vào trước list các đặc trưng để chúng ta có thể trích xuất cùng với những thứ khác:
    features_title = ['constant'] + features_title # đây là cách kết hợp 2 list
    # chia dữ liệu thành sub-DataFrame chứa các đặc trưng đã chỉ định (gồm constant)
    # gọi nó là features_columns.
    features_columns = data[features_title]

    # dòng tiếp theo sẽ trích xuất ma trận numpy từ biến features_columns:
    feature_matrix = features_columns.values
    # truy xuất dữ liệu được liên kết với đầu ra trong pandas Series
    # gọi nó là output_column
    output_column = data[labels_title]

    # tiếp theo sẽ chuyển đổi Series đã nhắc thành một mảng numpy
    output_array = output_column.values
    return(feature_matrix, output_array)

Cũng copy và paste cả hàm `predict_output()` để tính các dự đoán cho toàn bộ ma trận đặc trưng với ma trận và trọng số đã cho:

In [38]:
def predict_output(features, weights):
    # giả sử ma trận features chứa các đặc trưng ở dạng các cột và trọng số là mảng numpy tương ứng
    # tạo vectơ dự đoán sử dụng np.dot()
    predictions = np.dot(features, weights)
    return(predictions)

## Chuẩn hóa các đặc trưng
Trong tập dữ liệu giá nhà, các đặc trưng thay đổi khá nhiều về độ lớn: ví dụ, `sqft_living` rất lớn so với `bedrooms`. Do đó, trọng số cho `sqft_living` sẽ nhỏ hơn rất nhiều so với trọng số cho `bedrooms`. Điều này khó giải quyết vì các trọng số "nhỏ" bị giảm đầu tiên khi `l1_penalty` tăng. 

Để công bằng với tất cả các đặc trưng, chúng ta cần **chuẩn hóa đặc trưng** như đã thảo luận trong các bài giảng: chia mỗi đặc trưng cho chuẩn 2 của nó để đặc trưng đã biến đổi có chuẩn 1.

Hãy xem chúng ta có thể thực hiện chuẩn hóa này dễ dàng thế nào với Numpy: trước tiên chúng ta sẽ xem xét một ma trận nhỏ.

In [39]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print(X)

[[ 3.  5.  8.]
 [ 4. 12. 15.]]


Numpy cung cấp cách viết tắt để tính toán chuẩn 2 của mỗi cột:

In [40]:
norms = np.linalg.norm(X, axis=0) # cho [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print(norms)

[ 5. 13. 17.]


Để chuẩn hóa, hãy áp dụng phép chia element-wise (thực hiện trên các phần tử tương ứng):

In [41]:
print(X / norms) # cho [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.6        0.38461538 0.47058824]
 [0.8        0.92307692 0.88235294]]


Với viết tắt mà chúng ta vừa đề cập, hãy viết một hàm ngắn là `normalize_features(feature_matrix)`, hàm này chuẩn hóa các cột của ma trận đặc trưng đã cho. Hàm phải sẽ về một cặp `(normalized_features, norms)`, trong đó mục thứ hai chứa chuẩn của các đặc trưng ban đầu. Như đã thảo luận trong các bài giảng, chúng ta sẽ sử dụng các chuẩn này để chuẩn hóa dữ liệu kiểm tra theo cách mà chúng ta chuẩn hóa dữ liệu huấn luyện.

In [42]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    return (feature_matrix / norms, norms)

Để kiểm tra đạo hàm, chạy cell sau:

In [43]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print(features)
# sẽ in ra
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print(norms)
# sẽ in ra
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


## Triển khai Coordinate Descent với các đặc trưng được chuẩn hóa

Chúng ta tìm cách thu được một tập hợp trọng số thưa thớt bằng cách giảm thiểu hàm chi phí LASSO
<!-- ``` SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|). ``` -->
$cost = \sum (prediction - output)^2 + \lambda * \sum_{i \neq 0} |w_i|$

(Theo quy ước, chúng ta không bao hàm $w_0$ (độ chệch) trong phần tử L1 penalty. Chúng ta không bao giờ đẩy intercept thành 0.)

Dấu giá trị tuyệt đối làm cho hàm chi phí không thể phân biệt được, do đó gradient descent đơn giản không khả thi (bạn sẽ cần triển khai phương pháp subgradient descent). Thay vào đó, chúng ta sẽ sử dụng **coordinate descent**: ở mỗi lần lặp, chúng ta sẽ cố định tất cả các trọng số ngoại trừ trọng số `i` và tìm giá trị trọng số` i` thu nhỏ mục tiêu, tức là tìm: <br>
<!-- ``` argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ] ``` -->
$argmin_{w_i}(cost_i) = \sum (prediction - output)^2 + \lambda * \sum_{i \neq 0} |w_i| )$

trong đó tất cả các trọng số khác $w_i$(`w[i]`) được coi là không đổi. Chúng ta sẽ tối ưu hóa $w_i$ lần lượt, tuần hoàn nhiều lần qua các trọng số.
  1. Chọn tọa độ `i`
  2. Tính $w_i$ giảm thiểu hàm chi phí $cost = (\sum prediction - output) + \lambda * \sum_{i \neq 0} |w_i|$
  3. Lặp lại bước 1 và 2 cho tất cả các tọa độ nhiều lần. 

Với notebook này, chúng ta sử dụng **coordinate descent theo chu kỳ với các đặc trưng được chuẩn hóa**, trong đó chúng ta tuần hoàn qua các tọa độ theo thứ tự từ 0 đến (d-1) và giả sử các đặc trưng đã được chuẩn hóa như đã thảo luận ở trên. Công thức để tối ưu hóa từng tọa độ như sau:
<!-- ```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
``` -->
$$w_i = \left\{
\begin{array}{ll}
      \rho_i + \lambda / 2 & \rho_i < -\lambda/2 \\
      0 & -\lambda/2 \leq \rho_i \leq \lambda/2  \\
      \rho_i - \lambda / 2 & \rho_i > \lambda/2 \\
\end{array} 
\right. $$

trong đó $\rho_i$(`ro[i]`) được xác định như sau:
<!-- ```ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]. ``` -->
$\rho_i = \sum feature_i * (output - prediction + w_i*feature_i)$

Lưu ý là chúng ta không điều chuẩn trọng số của đặc trưng không đổi (intercept|độ chệch) $w_0$(`w[0]`), nên với trọng số này cập nhật đơn giản là:
<!-- ```w[0] = ro[i]``` -->
$w_0 = \rho_i$

## Tác động của L1 penalty

Xét mô hình đơn giản có 2 đặc trưng:

In [44]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(full_data, simple_features, my_output)

Đừng quên chuẩn hóa các đặc trưng:

In [45]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

Chúng ta gán một số tập hợp các trọng số ban đầu ngẫu nhiên và kiểm tra các giá trị của `ro[i]`:

In [46]:
weights = np.array([1., 4., 1.])

Sử dụng `predict_output()` để đưa ra dự đoán trên dữ liệu này. 

In [47]:
prediction = predict_output(simple_feature_matrix, weights)

Tính giá trị của `ro[i]` cho từng đặc trưng trong mô hình đơn giản này sử dụng công thức đã cho:
<!-- ```ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]. ``` -->
$\rho_i = \sum feature_i * (output - prediction + w_i*feature_i)$

*Gợi ý: có thể sử dụng vectơ Numpy cho feature_i bằng:*
```
simple_feature_matrix[:,i]
```

In [48]:
ro = []
for i in range(weights.size):
    ro_i = sum(simple_feature_matrix[:,i] * (output - prediction + weights[i] * simple_feature_matrix[:,i]))
    ro.append(ro_i)
    print("ro_{}: ".format(i) + ("%.2E" % ro_i))


ro_0: 7.94E+07
ro_1: 8.79E+07
ro_2: 8.10E+07


***QUIZ***

Nhớ lại rằng, bất cứ khi nào `ro[i]` nằm trong khoảng `-l1_penalty/2` và `l1_penalty/2` thì trọng số `w[i]` tương ứng sẽ về 0. Bây giờ, giả sử chúng ta thực hiện một bước coordinate descent ở đặc trưng 1 hoặc đặc trưng 2. Phạm vi giá trị nào của `l1_penalty`sẽ **không đặt** `w[1]` thành 0 mà **đặt** `w[2]` thành 0 nếu chúng ta lấy một bước trong tọa độ đó?

*Answer:*

$$\Leftrightarrow 
\left\{
\begin{array}{ll}
      \rho_1 \lt \frac{-\lambda}{2} & \rho_1 \gt \frac{\lambda}{2} \\
      \frac{-\lambda}{2} \leq \rho_2 \leq \frac{\lambda}{2} \\
\end{array} 
\right.
$$

<br>

$$
\Leftrightarrow 
\left\{
\begin{array}{ll}
      \lambda \lt  2\rho_1 \\
      \lambda \ge  2\rho_2 \\
\end{array} 
\right.
$$

<br>

$$
\Leftrightarrow 
2\rho_2 \le \lambda \lt 2\rho_1
$$







In [49]:
# Tài liệu quiz, hãy cẩn thận hơn
l1_penalty_low = 2 * ro[2]
l1_penalty_high = 2 * ro[1]
print("Range of l1_penalty: [" + ("%.2E" % l1_penalty_low) + "," + ("%.2E" % l1_penalty_high) + ")")


Range of l1_penalty: [1.62E+08,1.76E+08)


***QUIZ***

Phạm vi giá trị nào của `l1_penalty` sẽ đặt **cả** `w[1]` và `w[2]` thành 0 nếu lấy một bước trong tọa độ đó?

*Answer:*

$$
\Leftrightarrow 
\left\{
\begin{array}{ll}
      \frac{-\lambda}{2} \leq \rho_1 \leq \frac{\lambda}{2} \\
      \frac{-\lambda}{2} \leq \rho_2 \leq \frac{\lambda}{2} \\
\end{array} 
\right.
$$

<br>

$$
\Leftrightarrow 
\left\{
\begin{array}{ll}
      \lambda \ge 2\rho_1 \\
      \lambda \ge 2\rho_2 \\
\end{array} 
\right.
$$

Mà $\rho_1 \gt \rho_2 \Rightarrow \lambda \ge 2\rho_1 $, 

In [50]:
print('l1_penalty >=  ' + ("%.2E" % (2 * ro[1])))

l1_penalty >=  1.76E+08


Có thể nói rằng `ro[i]` định lượng tầm quan trọng của đặc trưng thứ i: `ro[i]` càng lớn thì đặc trưng thứ i càng có khả năng được giữ lại.

## Bước Coordinate Descent đơn lẻ

Hãy sử dụng công thức trên để triển khai coordinate descent giảm thiểu hàm chi phí trên đặc trưng đơn lẻ i. Lưu ý rằng intercept (trọng số 0) không được điều chuẩn. Hàm phải chấp nhận ma trận đặc trưng, đầu ra, trọng số hiện tại, l1 penalty và chỉ số của đặc trưng để tối ưu hóa hơn. Hàm sẽ trả về trọng số mới cho đặc trưng i.

In [51]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # tính prediction
    prediction = predict_output(feature_matrix, weights)
    # tính ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = sum(feature_matrix[:,i] * (output - prediction + weights[i] * feature_matrix[:,i]))

    if i == 0: # intercept -- không điều chuẩn
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

Hãy chạy cell sai để kiểm tra hàm:

In [52]:
# sẽ in ra 0.425558846691
import math
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1) )

0.4255588466910251


## Coordinate descent theo chu kỳ 

Bây giờ chúng ta có một hàm tối ưu hóa hàm chi phí trên một tọa độ duy nhất, hãy triển khai coordinate descent theo chu kỳ, trong đó tối ưu hóa tọa độ theo thứ tự 0, 1, ..., (d-1) và lặp lại.

Làm sao để biết khi nào thì dừng lại? Mỗi khi quét tất cả các tọa độ (đặc trưng) một lần, chúng ta sẽ đo lường thay đổi trọng số cho từng tọa độ. Nếu không có tọa độ nào thay đổi nhiều hơn một ngưỡng đã chỉ định thì dừng lại. 

Với mỗi lần lặp:
1. Khi lặp lại các đặc trưng theo thứ tự và thực hiện coordinate descent, hãy đo mức độ thay đổi của mỗi tọa độ.
2. Sau vòng lặp, nếu thay đổi lớn nhất trong tất cả các tọa độ nằm dưới dung sai thì hãy dừng lại. Nếu không, hãy quay lại bước 1.

Trả về trọng số

**QUAN TRỌNG: **khi tính trọng số mới cho tọa độ i, hãy đảm bảo tích hợp trọng số mới cho các tọa độ 0, 1, ..., i-1. Nên cập nhật biến trọng số tại chỗ. Xem minh họa với code giả sau.**
```python
với i trong range(len(weights)):
    old_weights_i = weights[i] # nhớ rằng giá trị cũ của weight[i] sẽ bị ghi đè
    # các dòng sau sử dụng giá trị mới cho weight[0], weight[1], ..., weight[i-1]
    #     và giá trị cũ cho weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # sử dụng old_weights_i để tính toán thay đổi về tọa độ
    ...
```

In [53]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    current_weights = initial_weights
    weight_changes = np.full(current_weights.size, tolerance)
    while np.max(weight_changes) >= tolerance:
        weight_changes = np.zeros(current_weights.size)
        for i in range(initial_weights.size):
            old_weight = current_weights[i]
            current_weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, current_weights, l1_penalty)
            weight_changes[i] = np.abs(current_weights[i] - old_weight)
        
        # print("Weights: " + str(current_weights))
        # print("Weight_changes: " + str(weight_changes))
    return current_weights

Sử dụng các tham số sau để tìm hiểu trọng số trong tập dữ liệu bán hàng. 

In [54]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

Trước tiên, hãy tạo phiên bản chuẩn hóa của ma trận đặc trưng  `normalized_simple_feature_matrix`.

In [55]:
(simple_feature_matrix, output) = get_numpy_data(full_data, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Sau đó chạy triển khai LASSO coordinate descent:

In [56]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
prediction = predict_output(normalized_simple_feature_matrix, weights)
rss = np.sum((output - prediction) ** 2)
print("{:.2E}".format(rss))
print(np.array(['constant'] + simple_features)[weights != 0])

1.63E+15
['constant' 'sqft_living']


***QUIZ***
1. RSS của mô hình đã nghiên cứu trong tập dữ liệu chuẩn hóa là bao nhiêu? (Gợi ý: sử dụng ma trận đặc trưng chuẩn hóa khi đưa ra dự đoán.)
   >> Answer: 1.63E+15
2. Đặc trưng nào có trọng số 0 khi hội tụ?
   >> Answer: 'bedrooms'

# Đánh giá khớp LASSO với nhiều đặc trưng hơn

Hãy chia dữ liệu thành tập huấn luyện và tập kiểm tra.

In [57]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(full_data, train_size=0.8, test_size=0.2, random_state=0)

Hãy xem xét tập đặc trưng sau.

In [58]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

Trước tiên, hãy tạo ma trận đặc trưng chuẩn hóa từ dữ liệu HUẤN LUYỆN với các đặc trưng này. (Lưu trữ các chuẩn cho việc chuẩn hóa vì chúng ta sẽ sử dụng chúng sau)

In [59]:
# sử dụng normalize_features.
features_matrix, output = get_numpy_data(train_data, all_features, 'price')
normalize_features_matrix, norms = normalize_features(features_matrix)
normalize_features_matrix

array([[0.00760506, 0.00652347, 0.00337304, ..., 0.00715153, 0.00748474,
        0.        ],
       [0.00760506, 0.00652347, 0.0084326 , ..., 0.00815274, 0.00763907,
        0.        ],
       [0.00760506, 0.00652347, 0.00505956, ..., 0.        , 0.00766607,
        0.        ],
       ...,
       [0.00760506, 0.00652347, 0.00758934, ..., 0.        , 0.00763135,
        0.        ],
       [0.00760506, 0.00869795, 0.00674608, ..., 0.        , 0.00763135,
        0.        ],
       [0.00760506, 0.00869795, 0.00758934, ..., 0.01201456, 0.00756576,
        0.        ]])

Trước tiên, hãy tìm hiểu trọng số với `l1_penalty=1e7` trong dữ liệu huấn luyện. Khởi tạo trọng số bằng 0, và đặt `tolerance=1`.  Gọi các trọng số kết quả `weights1e7`, chúng ta sẽ cần tới chúng sau.

In [60]:
l1_penalty = 1e7
tolerance = 1.0
initial_weights = np.zeros(len(all_features)+1)

weights1e7 = lasso_cyclical_coordinate_descent(normalize_features_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print(np.array(['constant'] + all_features)[weights1e7 != 0])
print("weights1e7: " + str(weights1e7))

['constant' 'sqft_living' 'waterfront' 'view']
weights1e7: [24625317.01587677        0.                0.         48292264.5359226
        0.                0.          3566000.33284842  7734349.09716171
        0.                0.                0.                0.
        0.                0.        ]


***QUIZ***

Trong trường hợp này, đặc trưng nào có trọng số khác 0?

>> Answer: ['constant' 'sqft_living' 'waterfront' 'view']

Tiếp theo, tìm hiểu trọng số với `l1_penalty=1e8` trong dữ liệu huấn luyện. Khởi tạo trọng số bằng 0 và đặt `tolerance=1`.  Gọi các trọng số kết quả `weights1e8`, chúng ta sẽ cần chúng sau.

In [61]:
weights1e8 = lasso_cyclical_coordinate_descent(normalize_features_matrix, output,
                                            initial_weights, 1e8, tolerance)
print(np.array(['constant'] + all_features)[weights1e8 != 0])
print("weights1e8: " + str(weights1e8))

['constant']
weights1e8: [71373534.790512        0.              0.              0.
        0.              0.              0.              0.
        0.              0.              0.              0.
        0.              0.      ]


***QUIZ***

Trong trường hợp này, đặc trưng nào có trọng số khác 0?

>> Answer: ['constant']

Cuối cùng, tìm hiểu trọng số với `l1_penalty=1e4` trong dữ liệu huấn luyện. Khởi tạo trọng số bằng 0 và đặt `tolerance=5e5`.  Gọi các trọng số kết quả `weights1e4`, chúng ta sẽ cần chúng sau..  (Trường hợp này sẽ cần thời gian [khá lâu](https://xkcd.com/303/) để hội tụ so với các trường hợp khác ở trên.)

In [62]:
# weights1e4 = lasso_cyclical_coordinate_descent(normalize_features_matrix, output,
#                                             initial_weights, 1e4, tolerance)
# print(np.array(['constant'] + all_features)[weights1e4 != 0])
# print("weights1e4: " + str(weights1e4))

In [70]:
# Chép lại chứ thời gian build lại là hơi đầu lâu
weights1e7 = [24625317.01587677, 0., 0., 48292264.5359226, 0., 0., 3566000.33284842, 7734349.09716171, 0. , 0. ,0., 0., 0., 0.]
weights1e8 = [71373534.790512, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0, 0.]
weights1e4 = [ 7.85539094e8, -1.67305993e7, 1.21699331e7, 5.25772449e7,-1.55699801e6, 4.91561288e6, 
              7.06846132e6, 4.89218766e6, 9.14020688e6, 1.23627056e8, 0., 5.89185945e4, -8.95125701e8, 6.02507234e5]


***QUIZ***

Trong trường hợp này, đặc trưng nào có trọng số khác 0?

>> Answer: 'constant' 'bedrooms' 'bathrooms' 'sqft_living' 'sqft_lot' 'floors' 'waterfront' 'view' 'condition' 'grade' 'sqft_basement' 'yr_built' 'yr_renovated'

## Tái tỷ lệ các trọng số đã nghiên cứu

Nhắc lại là chúng ta đã chuẩn hóa ma trận đặc trưng trước khi tìm hiểu các trọng số. Để sử dụng các trọng số này trên tập kiểm tra, cần chuẩn hóa dữ liệu kiểm tra theo cách tương tự.

Ngoài ra, chúng ta có thể tái tỷ lệ các trọng số đã học để đưa vào quá trình chuẩn hóa, do đó chúng ta không cần lo lắng về việc chuẩn hóa dữ liệu kiểm tra:

Trong trường hợp này, chúng ta cần tái tỷ lệ các trọng số kết quả để đưa ra các dự đoán với những đặc trưng *ban đầu*:
 1. Lưu chuẩn của các đặc trưng ban đầu vào vectơ `norms`:
```
features, norms = normalize_features(features)
```
 2. Chạy Lasso trên đặc trưng đã chuẩn hóa và thu được vectơ `weights` 
 3. Tính các trọng số cho các đặc trưng ban đầu bằng phép chia element-wise
```
weights_normalized = weights / norms
```
Giờ chúng ta có thể áp dụng `weights_normalized` cho dữ liệu kiểm tra mà không cần chuẩn hóa nó!

Tạo phiên bản chuẩn hóa của từng trọng số đã tìm hiểu ở trên. (`weights1e4`, `weights1e7`, `weights1e8`).

In [71]:
# Bạn đã lưu trữ các trọng số rồi chứ?
weights1e4_normalized = weights1e4 / norms
weights1e7_normalized = weights1e7 / norms
weights1e8_normalized = weights1e8 / norms

In [72]:
print("{:.4f}".format(weights1e7_normalized[3]))

161.2952


Để kiểm tra kết quả, nếu gọi `normalized_weights1e7` - phiên bản chuẩn hóa của `weights1e7` thì:
```
print("{:.4f}".format(normalized_weights1e7[3]))
```
sẽ trả về 161.2952

## Đánh giá từng mô hình đã nghiên cứu trên dữ liệu kiểm tra

Giờ chúng ta hãy đánh giá 3 mô hình trên dữ liệu kiểm tra: 

In [65]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Tính RSS của từng trọng số đã chuẩn hóa trong `test_feature_matrix`:

In [73]:
# Nếu cần nhiều cell hơn, Insert -> Insert Cell Below.
prediction = predict_output(test_feature_matrix, weights1e4_normalized)
rss = np.sum((test_output - prediction) ** 2)
print("RSS with weights1e4_normalized: {:.2E}".format(rss))

RSS with weights1e4_normalized: 1.83E+14


In [74]:
prediction = predict_output(test_feature_matrix, weights1e7_normalized)
rss = np.sum((test_output - prediction) ** 2)
print("RSS with weights1e7_normalized: {:.2E}".format(rss))

RSS with weights1e7_normalized: 2.72E+14


In [75]:
prediction = predict_output(test_feature_matrix, weights1e8_normalized)
rss = np.sum((test_output - prediction) ** 2)
print("RSS with weights1e8_normalized: {:.2E}".format(rss))

RSS with weights1e8_normalized: 5.15E+14


***QUIZ***

Mô hình nào thực hiện tốt nhất trên dữ liệu kiểm tra?

>> Answer: Mô hình với weights1e4 (l1_penalty=1e4) cho kết quả tốt nhất trên tập kiểm tra.